## Overview
Basic idea here is to use Tensorboard's t-SNE algorithm to view high dimensional data structure in real-time (i.e. you watch it try to split the features in high-dim space and map it to 2-d space). You use this notebook to get the dataset in a form Tensorboard needs, and then run this:

Run with: tensorboard --logdir=/home/ubuntu/Notebooks/project-tensorboard/log-1/ --port=6006

My video is available here:
https://tsne-audio-classification-viz.herokuapp.com/

In [1]:
import os
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle

In [2]:
PATH = os.getcwd()

In [3]:
LOG_DIR = PATH + '/project-tensorboard/log-1/'

In [4]:
with open('raw_df.pkl', 'rb') as f:
    audio_df = pickle.load(f)

In [64]:
features = audio_df.iloc[:, 0: -2]
labels = audio_df.iloc[:, -2]

In [76]:
labels = labels.replace({0:'AC',1:'Car Horn',2:'Children Playing',3:'Dog',4:'Drill',
                         5:'Engine Idle',6:'Gun Shot',7:'Jackhammer',8:'Siren',9:'Street Music'})

In [77]:
scaler = StandardScaler()
features = scaler.fit_transform(features)
features = pd.DataFrame(features)

In [81]:
tf_data = tf.Variable(features.values)

In [82]:
labels.to_csv('df_labels.tsv', sep='\t', encoding='utf8', header=True)

In [83]:
metadata = os.path.join(LOG_DIR, 'df_labels.tsv')

In [84]:
with tf.Session() as sess:
    saver = tf.train.Saver([tf_data])
    sess.run(tf_data.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'tf_data.ckpt'))
    config = projector.ProjectorConfig()
    
    embedding = config.embeddings.add()
    embedding.tensor_name = tf_data.name
    
    embedding.metadata_path = metadata
    
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)